In [1]:
from pathlib import Path

from collections import defaultdict

import numpy as np

from itertools import combinations

from dataclasses import dataclass

import math

import pickle

from csp import Constraint, CSP

In [2]:
data_path = Path.home() / 'workstation' / 'dev' / 'Advent-of-Code-2020' / 'data' / 'day20_input.txt'

In [3]:
data_path.exists()

True

In [4]:
with open(data_path, 'r') as reader:
    image_input = reader.read().strip()

In [5]:
imgs_w_title = image_input.split('\n\n')

In [6]:
def get_pixels(text): 
    "Translate the '.' and '#' to 0's and 1's"
    bits = {'#': 1, '.': 0}
    return tuple(bits[p] for p in text)

def rotate_ctrclockwise(array, **kwargs):
    return np.rot90(array, **kwargs)

def flip_horizontal(array):
    return np.fliplr(array)

def flip_vertical(array):
    return np.flipud(array)

def get_tile_borders(tile_array):
    return [
        tile_array[:, 0],
        tile_array[:, -1],
        tile_array[0, :],
        tile_array[-1, :]
    ]

In [7]:
image_dict = {}

for image in imgs_w_title:
    title, *image = image.split('\n')
    title = title[:-1] # Getting rid of colon
    tile_id = [int(s) for s in title.split() if s.isdigit()]
    image_dict[tile_id[0]] = np.asarray(list(get_pixels(row) for row in image))

In [8]:
# All possible tile borders. 8 possibilities: original borders + flips

tile_borders = defaultdict(list)

for tile_id, tile in image_dict.items():
    tile_borders[tile_id] = (
        get_tile_borders(tile) +
        list(map(flip_vertical, get_tile_borders(tile)))
    )

In [9]:
common_borders = defaultdict(list)

for id_1, id_2 in combinations(tile_borders.keys(), 2):
    cmn_borders = []
    for bdr in tile_borders[id_1]:
        if any(np.array_equal(bdr, x) for x in tile_borders[id_2]):
            cmn_borders.append(bdr)
    common_borders[(id_1, id_2)] = cmn_borders
    common_borders[(id_2, id_1)] = cmn_borders

#### Part 2

#### Getting the right tiles for each cell via CSP

In [10]:
class TilingConstraint(Constraint):
    
    def __init__(self, cell1, cell2):
        super().__init__([cell1, cell2])
        self.cell1 = cell1
        self.cell2 = cell2
    
    def is_satisfied(self, assignment):
        def is_nbr(dx, dy):
            if dx == 1 and dy == 0:
                return True
            elif dx == 0 and dy == 1:
                return True
            else: return False
            
        if self.cell1 not in assignment or self.cell2 not in assignment:
            return True
        
        x1, y1 = self.cell1.row, self.cell1.col
        x2, y2 = self.cell2.row, self.cell2.col
        
        dx = abs(x1 - x2)
        dy = abs(y1 - y2)
        
        if is_nbr(dx, dy):
            if TilingConstraint.border_pairs[(assignment[self.cell1], assignment[self.cell2])]:
                return True
            else: return False
        else:
            if not TilingConstraint.border_pairs[(assignment[self.cell1], assignment[self.cell2])]:
                return True
            else: return False

In [11]:
setattr(TilingConstraint, 'border_pairs', common_borders)

In [12]:
@dataclass(eq=True, frozen=True)
class GridLocation:
    '''Object for tracking cell location'''
    row: int 
    col: int

In [13]:
def generate_grid(num_rows, num_columns):
    return [GridLocation(i, j) for i in range(num_rows) for j in range(num_columns)]

In [14]:
len_sides = int(math.sqrt(len(imgs_w_title)))

In [15]:
variables = generate_grid(len_sides, len_sides)

In [16]:
domains = {}

for variable in variables:
    domains[variable] = list(image_dict.keys())

csp = CSP(variables, domains)

In [17]:
for cell_1, cell_2 in combinations(variables, 2):
    csp.add_constraint(TilingConstraint(cell_1, cell_2))

In [20]:
solution = csp.backtracking_search_unique_assignment()

Creating an intermediate save file as the previous operation is a bit time-consuming

Search process not really optimized.

In [21]:
# Saving results in a file

with open('Grid_TileIds.txt', 'wb') as handle:
    pickle.dump(solution, handle)

In [18]:
# Loading results from a file

with open('Grid_TileIds.txt', 'rb') as handle:
    tile_dict = pickle.loads(handle.read())

#### Getting the right orientation for each tile via CSP

In [19]:
def get_all_possible_orientations(image_arr):
    subimage_list = []
    for rot in range(4):
        sub_image = rotate_ctrclockwise(
            image_arr,
            k=rot
        )
        subimage_list.append(sub_image)
        subimage_list.append(flip_horizontal(sub_image))
        subimage_list.append(flip_vertical(sub_image))
    return subimage_list

In [20]:
domains = defaultdict(list)

for variable in variables:
    image = image_dict[tile_dict[variable]]
    domains[variable] = get_all_possible_orientations(image)

In [21]:
class OrientationConstraint(Constraint):
    def __init__(self, cell1, cell2,):
        super().__init__([cell1, cell2])
        self.cell1 = cell1
        self.cell2 = cell2
        
        
    def is_satisfied(self, assignment):            
        if self.cell1 not in assignment or self.cell2 not in assignment:
            return True
        
        y1, x1 = self.cell1.row, self.cell1.col
        y2, x2 = self.cell2.row, self.cell2.col
        
        img1 = assignment[self.cell1]
        img2 = assignment[self.cell2]
        
        if x2 == x1+1 and y2 == y1:
            if np.array_equal(img1[:, -1], img2[:, 0]):
                return True
            else:
                return False
        elif x2 == x1-1 and y2 == y1:
            if np.array_equal(img1[:, 0], img2[:, -1]):
                return True
            else:
                return False
        elif y2 == y1+1 and x2 == x1:
            if np.array_equal(img1[-1, :], img2[0, :]):
                return True
            else:
                return False
        elif y2 == y1-1 and x2 == x1:
            if np.array_equal(img1[0, :], img2[-1, :]):
                return True
            else:
                return False
        else:
            return True

In [22]:
csp = CSP(variables, domains)

In [23]:
for cell_1, cell_2 in combinations(variables, 2):
    csp.add_constraint(OrientationConstraint(cell_1, cell_2))

In [24]:
solution = csp.backtracking_search()

In [25]:
def get_final_image(solution, len_sides):
    final_image= []
    for i in range(len_sides):
        row = []
        for j in range(len_sides):
            image = solution[GridLocation(i, j)][1:-1, 1:-1]
            row.append(image)
        final_image.append(row)
    return np.bmat(final_image)

In [26]:
final_image = get_final_image(solution, len_sides)

In [28]:
final_image_list = get_all_possible_orientations(final_image)

In [29]:
def count_non_monster_ones(image):
    def is_monster_present(arr):
        conditions = (
            arr[0, 18] == 1 and
            arr[1, 0]  == 1 and
            arr[1, 5]  == 1 and
            arr[1, 6]  == 1 and
            arr[1, 11] == 1 and
            arr[1, 12] == 1 and
            arr[1, 17] == 1 and
            arr[1, 18] == 1 and
            arr[1, 19] == 1 and
            arr[2, 1]  == 1 and
            arr[2, 4]  == 1 and
            arr[2, 7]  == 1 and
            arr[2, 10] == 1 and
            arr[2, 13] == 1 and
            arr[2, 16] == 1
        )
    
        if conditions:
            return True
        else:
            return False
        
        
    def search_and_count_monsters():
        count = 0
        for i in range(0, len_sides-3):
            for j in range(0, len_sides-20):
                search_array = image[i: i+3, j: j+20]
                if is_monster_present(search_array):
                    count += 1
        return count
    
    
    len_sides = image.shape[0]
    num_monsters = search_and_count_monsters()
    num_non_monster_ones = image.sum() - num_monsters*15
    return num_non_monster_ones 

In [30]:
for index, image in enumerate(final_image_list):
    if count_non_monster_ones(image) < image.sum():
        print(count_non_monster_ones(image))

1964
